In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [27]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split

tf.__version__

'2.1.0'

# Introducing convolutional neural networks

# Classifying images

## Using one-hot encoding to represent images

In [3]:
# given a set of labels
labels = ['shoe', 'shirt', 'shoe', 'shirt', 'dress', 'dress', 'dress']

In [4]:
# the number of image categories
n_categories = 3

# the unique valies of categories in the data
categories = np.array(['shirt', 'dress', 'shoe'])

# initialize ohe_labels as all zeros

ohe_labels = np.zeros((len(labels), n_categories))

# loop over the labels and encode
for ii in range(len(labels)):
    
    # find the location of this label in the categories
    jj = np.where(labels[ii]==categories)
    
    # set the corresponding zero to one
    ohe_labels[ii, jj] = 1

## Evaluating a classifier

In [5]:
test_labels = np.array([[0., 0., 1.],
                        [0., 1., 0.],
                        [0., 0., 1.],
                        [0., 1., 0.],
                        [0., 0., 1.],
                        [0., 0., 1.],
                        [0., 0., 1.],
                        [0., 1., 0.]])

preds = np.array([[0., 0., 1.],
                  [0., 1., 0.],
                  [0., 0., 1.],
                  [1., 0., 0.],
                  [0., 0., 1.],
                  [1., 0., 0.],
                  [0., 0., 1.],
                  [0., 1., 0.]])

In [6]:
# calculate the number of correct predictions
number_correct = (test_labels*preds).sum()
print(number_correct)

proportion_correct = number_correct / len(preds)
print(proportion_correct)

6.0
0.75


# Classification with `keras`

## Build a neural network

In [24]:
# let's just use the 10k images from the fashion mnist dataset
(_, _), (X, y) = tf.keras.datasets.fashion_mnist.load_data()

In [25]:
X = X.reshape(X.shape[0], X.shape[1]*X.shape[2])
y = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.1,
                                                    stratify=y,
                                                    random_state=42)

In [48]:
# define model arch
model = Sequential()

model.add(Dense(50, input_shape=(784,), activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer=Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [50]:
# fit model
early_stop = EarlyStopping(monitor='val_accuracy',
                           patience=5)

model.fit(X_train, y_train,
          epochs=100,
          batch_size=256,
          validation_split=0.2,
          callbacks=[early_stop],
          verbose=2)

Train on 7200 samples, validate on 1800 samples
Epoch 1/100
7200/7200 - 0s - loss: 0.6335 - accuracy: 0.7526 - val_loss: 0.9797 - val_accuracy: 0.6917
Epoch 2/100
7200/7200 - 0s - loss: 0.6403 - accuracy: 0.7504 - val_loss: 0.9541 - val_accuracy: 0.7233
Epoch 3/100
7200/7200 - 0s - loss: 0.6221 - accuracy: 0.7543 - val_loss: 0.9060 - val_accuracy: 0.7467
Epoch 4/100
7200/7200 - 0s - loss: 0.6082 - accuracy: 0.7608 - val_loss: 0.9117 - val_accuracy: 0.7422
Epoch 5/100
7200/7200 - 0s - loss: 0.6141 - accuracy: 0.7614 - val_loss: 0.9444 - val_accuracy: 0.7100
Epoch 6/100
7200/7200 - 0s - loss: 0.6168 - accuracy: 0.7518 - val_loss: 0.9442 - val_accuracy: 0.7150
Epoch 7/100
7200/7200 - 0s - loss: 0.6014 - accuracy: 0.7604 - val_loss: 0.9472 - val_accuracy: 0.7222
Epoch 8/100
7200/7200 - 0s - loss: 0.5816 - accuracy: 0.7733 - val_loss: 0.8943 - val_accuracy: 0.7517
Epoch 9/100
7200/7200 - 0s - loss: 0.5901 - accuracy: 0.7651 - val_loss: 0.9352 - val_accuracy: 0.7300
Epoch 10/100
7200/7200 - 

In [52]:
# eval
print(f'test acc: {model.evaluate(X_test, y_test)[1]: .1%}')

1000/1000 [==============================] - 0s 421us/sample - loss: 0.8736 - accuracy: 0.7280
test acc:  72.8%
